In [12]:
# Import numpy and OpenCV
import numpy as np
import cv2

In [23]:
def movingAverage(curve, radius): 
    window_size = 2 * radius + 1
    # Define the filter 
    f = np.ones(window_size)/window_size 
    # Add padding to the boundaries 
    curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
    # Apply convolution 
    curve_smoothed = np.convolve(curve_pad, f, mode='same') 
    # Remove padding 
    curve_smoothed = curve_smoothed[radius:-radius]
    # return smoothed curve
    return curve_smoothed 

def smooth(trajectory): 
    smoothed_trajectory = np.copy(trajectory) 
    # Filter the x, y and angle curves
    for i in range(3):
        smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)

    return smoothed_trajectory

def fixBorder(frame):
    s = frame.shape
    # Scale the image 4% without moving the center
    T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
    frame = cv2.warpAffine(frame, T, (s[1], s[0]))
    return frame

In [24]:
VIDEO_PATH = "../videos/"
INPUT_FILEPATH = VIDEO_PATH + "RPI Men's Hockey vs. Cornell University.mp4"
OUTPUT_FILEPATH = VIDEO_PATH + "AAA.mp4"

In [28]:
# The larger the more stable the video, but less reactive to sudden panning
SMOOTHING_RADIUS=5 

# Read input video
cap = cv2.VideoCapture(INPUT_FILEPATH) 
 
# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
 
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get frames per second (fps)
fps = cap.get(cv2.CAP_PROP_FPS)
 
# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
 
# Set up output video
out = cv2.VideoWriter(OUTPUT_FILEPATH, fourcc, fps, (2 * w, h))

# Read first frame
_, prev = cap.read() 
 
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY) 

# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 

for i in range(n_frames-2):
  # Detect feature points in previous frame
  prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)
   
  # Read next frame
  success, curr = cap.read() 
  if not success: 
    break 

  # Convert to grayscale
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 

  # Calculate optical flow (i.e. track feature points)
  curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 

  # Sanity check
  assert prev_pts.shape == curr_pts.shape 

  # Filter only valid points
  idx = np.where(status==1)[0]
  prev_pts = prev_pts[idx]
  curr_pts = curr_pts[idx]

  #Find transformation matrix
  m = cv2.estimateRigidTransform(prev_pts, curr_pts, fullAffine=False) #will only work with OpenCV-3 or less
   
  # Extract traslation
  dx = m[0,2]
  dy = m[1,2]

  # Extract rotation angle
  da = np.arctan2(m[1,0], m[0,0])
   
  # Store transformation
  transforms[i] = [dx,dy,da]
   
  # Move to next frame
  prev_gray = curr_gray

  print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/523 -  Tracked points : 190
Frame: 1/523 -  Tracked points : 196
Frame: 2/523 -  Tracked points : 186
Frame: 3/523 -  Tracked points : 199
Frame: 4/523 -  Tracked points : 192
Frame: 5/523 -  Tracked points : 196
Frame: 6/523 -  Tracked points : 194
Frame: 7/523 -  Tracked points : 198
Frame: 8/523 -  Tracked points : 198
Frame: 9/523 -  Tracked points : 198
Frame: 10/523 -  Tracked points : 198
Frame: 11/523 -  Tracked points : 198
Frame: 12/523 -  Tracked points : 197
Frame: 13/523 -  Tracked points : 198
Frame: 14/523 -  Tracked points : 200
Frame: 15/523 -  Tracked points : 197
Frame: 16/523 -  Tracked points : 197
Frame: 17/523 -  Tracked points : 197
Frame: 18/523 -  Tracked points : 199
Frame: 19/523 -  Tracked points : 197
Frame: 20/523 -  Tracked points : 195
Frame: 21/523 -  Tracked points : 197
Frame: 22/523 -  Tracked points : 197
Frame: 23/523 -  Tracked points : 199
Frame: 24/523 -  Tracked points : 199
Frame: 25/523 -  Tracked points : 199
Frame: 26/523 -  Track

Frame: 217/523 -  Tracked points : 198
Frame: 218/523 -  Tracked points : 200
Frame: 219/523 -  Tracked points : 200
Frame: 220/523 -  Tracked points : 200
Frame: 221/523 -  Tracked points : 200
Frame: 222/523 -  Tracked points : 200
Frame: 223/523 -  Tracked points : 200
Frame: 224/523 -  Tracked points : 199
Frame: 225/523 -  Tracked points : 200
Frame: 226/523 -  Tracked points : 200
Frame: 227/523 -  Tracked points : 200
Frame: 228/523 -  Tracked points : 199
Frame: 229/523 -  Tracked points : 200
Frame: 230/523 -  Tracked points : 199
Frame: 231/523 -  Tracked points : 199
Frame: 232/523 -  Tracked points : 199
Frame: 233/523 -  Tracked points : 196
Frame: 234/523 -  Tracked points : 200
Frame: 235/523 -  Tracked points : 199
Frame: 236/523 -  Tracked points : 200
Frame: 237/523 -  Tracked points : 199
Frame: 238/523 -  Tracked points : 199
Frame: 239/523 -  Tracked points : 199
Frame: 240/523 -  Tracked points : 199
Frame: 241/523 -  Tracked points : 200
Frame: 242/523 -  Tracked

Frame: 429/523 -  Tracked points : 153
Frame: 430/523 -  Tracked points : 156
Frame: 431/523 -  Tracked points : 141
Frame: 432/523 -  Tracked points : 152
Frame: 433/523 -  Tracked points : 152
Frame: 434/523 -  Tracked points : 156
Frame: 435/523 -  Tracked points : 149
Frame: 436/523 -  Tracked points : 142
Frame: 437/523 -  Tracked points : 151
Frame: 438/523 -  Tracked points : 146
Frame: 439/523 -  Tracked points : 149
Frame: 440/523 -  Tracked points : 147
Frame: 441/523 -  Tracked points : 142
Frame: 442/523 -  Tracked points : 151
Frame: 443/523 -  Tracked points : 145
Frame: 444/523 -  Tracked points : 146
Frame: 445/523 -  Tracked points : 147
Frame: 446/523 -  Tracked points : 144
Frame: 447/523 -  Tracked points : 144
Frame: 448/523 -  Tracked points : 148
Frame: 449/523 -  Tracked points : 146
Frame: 450/523 -  Tracked points : 140
Frame: 451/523 -  Tracked points : 139
Frame: 452/523 -  Tracked points : 140
Frame: 453/523 -  Tracked points : 134
Frame: 454/523 -  Tracked

In [29]:

# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0) 
 
# Create variable to store smoothed trajectory
smoothed_trajectory = smooth(trajectory) 

# Calculate difference in smoothed_trajectory and trajectory
difference = smoothed_trajectory - trajectory
 
# Calculate newer transformation array
transforms_smooth = transforms + difference

# Reset stream to first frame 
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
 
# Write n_frames-1 transformed frames
for i in range(n_frames-2):
  # Read next frame
  success, frame = cap.read() 
  if not success:
    break

  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]

  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy

  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (w,h))

  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 

  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])

  # If the image is too big, resize it.
  if(frame_out.shape[1] > 1920): 
    frame_out = cv2.resize(frame_out, (int(frame_out.shape[1]/2), int(frame_out.shape[0]/2)));
  
  cv2.imshow("Before and After", frame_out)
  cv2.waitKey(10)
  out.write(frame_out)

# Release video
cap.release()
out.release()
# Close windows
cv2.destroyAllWindows()